In [160]:
!pip install -r requirements.txt
!pip install tabulate
import numpy as np
import sys
from tabulate import tabulate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [161]:
from PIL import Image

def save_images_as_png(images_array, prefix="image"):
    for i, image_array in enumerate(images_array):
        # Convert the image array to PIL Image
        image = Image.fromarray(image_array)
        # Save the image as PNG with a sequential filename
        filename = f"{prefix}_{i}.png"
        image.save(f"images/{filename}")

def read_images_from_binary_file(path):
    with open(path,"rb") as file:
        magic_number=file.read(4)
        number_of_items = int.from_bytes(file.read(4), byteorder='big')
        number_of_rows = int.from_bytes(file.read(4), byteorder='big')
        number_of_columns = int.from_bytes(file.read(4), byteorder='big')
        image_data=file.read()
        size=number_of_rows*number_of_columns

    images_array=np.frombuffer(image_data,dtype=np.uint8)
    images_array = images_array.reshape((number_of_items, number_of_columns, number_of_rows))
    return images_array

def read_training_set_label_file(filename):
    with open(filename, 'rb') as file:
        # Read the magic number and other metadata
        magic_number = int.from_bytes(file.read(4), byteorder='big')
        num_labels = int.from_bytes(file.read(4), byteorder='big')

        # Read the label data
        label_data = np.frombuffer(file.read(num_labels), dtype=np.uint8)

    return label_data

training_data=read_images_from_binary_file("data/train-images.idx3-ubyte")
training_label=read_training_set_label_file("data/train-labels.idx1-ubyte")
testing_data=read_images_from_binary_file("data/t10k-images.idx3-ubyte")
testing_label=read_training_set_label_file("data/train-labels.idx1-ubyte")



In [162]:
learning_rate=1
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sigmoid_x = sigmoid(x)
    return sigmoid_x * (1 - sigmoid_x)

class Layer:
   def __init__(self, Values):
      self.values = Values
      self.zvalues= None
      self.weights = None
      self.biases = None
      self.delta_weights= None
      self.delta_biases = None
      self.next_layer=None
      self.previous_layer=None
      
      

   def nextLayer(self,nodes,last_layer=False):
      if nodes!=0:
         self.weights = np.random.rand(self.values.size, nodes) * 0.01
         self.biases = np.random.rand(nodes) * 0.01
         self.delta_weights = np.random.rand(self.values.size, nodes)
         self.delta_biases = np.random.rand(nodes)         
         self.delta_values=np.zeros(self.values.size)
         self.delta_zvalues=np.zeros(self.values.size)

      values= np.dot(self.values.reshape(1, self.values.size),self.weights).flatten() + self.biases
      zvalues=values
      for i,value in enumerate(values):
         value=sigmoid(value)
         values[i]=value

      layer=Layer(values)
      self.next_layer=layer
      layer.zvalues=zvalues
      layer.previous_layer=self
      if last_layer==True:
         layer.delta_values=np.zeros(nodes)
         layer.delta_zvalues=np.zeros(nodes)
      return layer
    
    
    
   def forward(self,image):
      layer = self
      while layer.next_layer is not None:
         print("Weights Shape:", layer.weights.shape, 
               "| Biases Shape:", layer.biases.shape,
               "| Delta Weights Shape:", layer.delta_weights.shape,
               "| Delta Biases Shape:", layer.delta_biases.shape,
               "| Delta Values Shape:", layer.delta_values.shape,
               "| Delta Z Values Shape:", layer.delta_zvalues.shape)
         layer = layer.next_layer
         
      layer=self
      layer.values=image
      while layer.next_layer!=None:
         zvalues=np.dot(layer.values.reshape(1, layer.values.size),layer.weights).flatten() +layer.biases
         layer.next_layer.zvalues=zvalues
         for i,value in enumerate(zvalues):
            zvalues[i]=sigmoid(value)
         layer.next_layer.values=zvalues
         layer=layer.next_layer
      

    
   def backward(self,expectation):
      layer=self
      while layer.next_layer!=None:
         layer=layer.next_layer
      for i,value in enumerate(layer.values):
         layer.delta_values[i]=2*(value-expectation[i])
         layer.delta_zvalues[i]=layer.delta_values[i] * sigmoid_derivative(layer.zvalues[i])
         layer.delta_biases=np.copy(layer.delta_zvalues)

      layer=layer.previous_layer
      while layer.previous_layer!=None:
         for i, row in enumerate(layer.weights):
            for j, value in enumerate(row):
               layer.delta_weights[i][j]=layer.previous_layer.delta_zvalues[j] * layer.values[i]

         for i,value in enumerate(layer.values):
            new_val=0
            print(layer.values.shape, layer.next_layer.delta_zvalues.shape,layer.weights.shape)
            for j,value in enumerate(layer.next_layer.delta_zvalues):
               new_val=new_val+value * layer.weights[i][j]

            layer.delta_values[i]=new_val


         for i,value in enumerate(layer.zvalues):
            layer.delta_zvalues=layer.delta_values[i] * sigmoid_derivative(layer.zvalues[i])

         layer.delta_biases=np.copy(layer.delta_zvalues)
         layer=layer.previous_layer
      
      layer=self
      while layer.next_layer!=None:
         layer=layer.next_layer

      while layer.previous_layer!=None:
         layer.biases=layer.biases-layer.delta_biases
         layer.weights=layer.weights-layer.delta_weights
         layer=layer.previous_layer
      
   def outcome(self):
      layer=self
      while layer.next_layer!=None:
         layer=layer.next_layer
      return layer.values
    
   
    

In [163]:


def costFunction(actual, expectation):
    cost=None
    for i, value in enumerate(actual):
        cost=cost+(value-expectation[i])**2
    return cost

def digit(number):
    arr=np.zeros(10)
    arr[number]=1
    return arr

def number_from_digit(arr):
    return np.argmax(arr)

sample_image=training_data[0]
first_layer=Layer(sample_image)
second_layer=first_layer.nextLayer(300)
third_layer=second_layer.nextLayer(100)
fourth_layer=third_layer.nextLayer(10,last_layer=True)
first_layer.forward(training_data[0])


for i,Image in enumerate(training_data):
    first_layer.forward(Image)
    print(i)
    first_layer.backward(digit(training_label[i]))

first_layer(training_data[20])
print(first_layer.outcome)

Weights Shape: (784, 300) | Biases Shape: (300,) | Delta Weights Shape: (784, 300) | Delta Biases Shape: (300,) | Delta Values Shape: (784,) | Delta Z Values Shape: (784,)
Weights Shape: (300, 100) | Biases Shape: (100,) | Delta Weights Shape: (300, 100) | Delta Biases Shape: (100,) | Delta Values Shape: (300,) | Delta Z Values Shape: (300,)
Weights Shape: (100, 10) | Biases Shape: (10,) | Delta Weights Shape: (100, 10) | Delta Biases Shape: (10,) | Delta Values Shape: (100,) | Delta Z Values Shape: (100,)
Weights Shape: (784, 300) | Biases Shape: (300,) | Delta Weights Shape: (784, 300) | Delta Biases Shape: (300,) | Delta Values Shape: (784,) | Delta Z Values Shape: (784,)
Weights Shape: (300, 100) | Biases Shape: (100,) | Delta Weights Shape: (300, 100) | Delta Biases Shape: (100,) | Delta Values Shape: (300,) | Delta Z Values Shape: (300,)
Weights Shape: (100, 10) | Biases Shape: (10,) | Delta Weights Shape: (100, 10) | Delta Biases Shape: (10,) | Delta Values Shape: (100,) | Delta

TypeError: 'numpy.float64' object is not iterable